In [5]:
'''
希科智慧病房：住院护士交班系统，交班项推荐
      —宽表设计：
          病人基础信息(patientID, Inhostimes, sex, age, nurselevel, Diagnosis)
          病情信息(Signsname,Signsvalue)+交班内容(dutycontent)+日期时间(datetime)
      —需求场景：根据病人当前的病情信息推荐相似的交班内容；
      —算法设计：文本匹配->时序建模->基于内容推荐； 
'''

In [92]:
import pandas as pd
import numpy as np
import string
#测试数据_医嘱护理测试数据
data = pd.read_csv("/data/fjsdata/nursereport/patient_times.csv", sep=',', header=None, \
                   names=['pId','inNum','sex','age','nrLevel','diagCon1','signName','signValue','diagCon2','dutyCon','datetime'], \
                   low_memory=False, encoding='GBK')
print ('Dataset shape of month 6 is:%d rows and %d columns'%(data.shape[0],data.shape[1]))
#dutyCon空值和缺失值处理
data['dutyCon']=data['dutyCon'].apply(lambda x: 0 if str(x).strip()=='' else x)#空值填充
data['dutyCon']=data['dutyCon'].fillna(0)#缺失值填充
'''
#signName和signVlue 拼接，diagCon1和diagCon2 拼接，signInfo 和 diagCon 拼接成病情描述
data['signInfo']=data['signName'].str.cat(data['signValue'],sep=',')
data['diagCon']=data['diagCon1'].str.cat(data['diagCon2'],sep=',')
data['conCon']=data['diagCon'].str.cat(data['signInfo'],sep=',')
#取diagInfo和dutyCon匹配
data=data[['pId','conCon','dutyCon','datetime']]
txtData = []
for pid in list(set(data['pId'].tolist())):
    df = data[data['pId']==pid]
    #df=df.sort_values(by='datetime')
    conCon=''
    for index,row in df.iterrows():
        if row['dutyCon']==0:#无交班内容
            conCon=conCon+','+str(row['conCon']).strip()#拼接
        else:#有交班内容
            txtData.append(conCon+','+row['dutyCon'])
            conCon=''
with open('/data/comcode/SentenceSimilarity/data/med.txt', 'w', encoding='gbk') as f:
    for item in txtData:
        f.write("%s\n" % item)
'''
data=data[data['dutyCon']!=0]
data=data['dutyCon']
print (data.shape)
data.to_csv('/data/tmpexec/med.csv',encoding='utf-8',index=False)

Dataset shape of month 6 is:83137 rows and 11 columns
(3451,)


In [67]:
# coding:utf-8  
'''
Created on 2019年7月2日

@author: cvter
'''
import argparse
import jieba
from gensim.models import doc2vec
import csv

def parse_args():#define the paramter of program
    parser = argparse.ArgumentParser(description="dutyModel.")
    parser.add_argument('--fileName', nargs='?', default='med.csv',help='fileName.')
    return parser.parse_args(args=[])

def get_stop_words():#load the stopwords 
    spath = '../data/stopword.txt'
    stopwords = [line.strip() for line in open(spath, 'r', encoding='GBK').readlines()]  
    return stopwords

def get_lineText(textpath): #get the data and tokenize
    rows = csv.reader(open(textpath,'r',encoding='utf-8'))
    lineText = []
    rawText = []
    stopwords = get_stop_words()
    for r in rows:
        rawText.append(r[0])
        seg_list = jieba.lcut(r[0].strip()) 
        txt_list = [' '.join(seg) for seg in seg_list if seg not in stopwords]
        lineText.append(txt_list)
    return lineText,rawText

def train_doc2vec_model(tagged_data):
    max_epochs = 100
    vec_size = 20
    alpha = 0.025
    #If dm=1 means ‘distributed memory’ (PV-DM) and dm =0 means ‘distributed bag of words’ (PV-DBOW). 
    model = doc2vec.Doc2Vec(size=vec_size,alpha=alpha, min_alpha=0.00025,min_count=1,dm =1)
    model.build_vocab(tagged_data)
    for epoch in range(max_epochs):
        print('iteration {0}'.format(epoch))
        model.train(tagged_data,total_examples=model.corpus_count,epochs=model.iter)
        # decrease the learning rate
        model.alpha -= 0.0002
        # fix the learning rate, no decay
        model.min_alpha = model.alpha
    
    model.save("../data/d2v.model")
    print("Doc2Vec Model Saved")

if __name__ == '__main__':
    #1.读取参数
    args = parse_args()
    fileName = args.fileName
    #2.加载文本，utf-8格式
    lineText,rawText = get_lineText('../data/'+fileName)
    print ("The texts %d has been loaded successfully in the file %s" % (len(lineText),fileName))
    with open('../data/raw.txt','w') as fw:
        lists=[line+"\n" for line in rawText]
        fw.writelines(lists)
    #3.词典生成和训练模型
    tagged_data = [doc2vec.TaggedDocument(words=line, tags=[str(i)]) for i, line in enumerate(lineText)]
    train_doc2vec_model(tagged_data)

In [ ]:
# coding:utf-8  
'''
Created on 2019年7月2日

@author: cvter
'''
import argparse
import jieba
from gensim.models import doc2vec

def parse_args():#define the paramter of program
    parser = argparse.ArgumentParser(description="dutySim.")
    parser.add_argument('--text', nargs='?', default='患者神志清 , 精神差 , 心电监测示:窦性心率 , 律齐。',help='text.')
    parser.add_argument('--topK', nargs='?', default='10',help='topK.')
    return parser.parse_args(args=[])

def get_stop_words():#load the stopwords 
    spath = '../data/stopword.txt'
    stopwords = [line.strip() for line in open(spath, 'r', encoding='GBK').readlines()]  
    return stopwords

def get_lineText(text): #get the data and tokenize
    stopwords = get_stop_words()
    seg_list = jieba.lcut(text.strip()) 
    lineText = [' '.join(seg) for seg in seg_list if seg not in stopwords]
    return lineText

def get_most_similar(lineText,topK):
    model= doc2vec.Doc2Vec.load("../data/d2v.model")
    predVec = model.infer_vector(lineText)
    '''
    mostSim=model.docvecs.most_similar(0)
    sims = model.docvecs.similarity(1,2)#计算两两相似度
    docvec =model.docvecs[1]#返回对应的向量
    Returns:    Sequence of (doctag/index, similarity).
    Return type:    list of ({str, int}, float)
    '''
    mostSim=model.docvecs.most_similar([predVec], topn=int(topK))

    return mostSim

if __name__ == '__main__':
    #1.读取参数
    args = parse_args()
    text = args.text
    topK = args.topK
    #2.分词
    lineText = get_lineText(text)
    #3.匹配相似度最高k项
    mostSim = get_most_similar(lineText,topK)
    rawText = [line for line in open('../data/raw.txt', 'r').readlines()]
    simItems = []
    for i, sim in mostSim:
        strtxt = rawText[int(i)]
        simItems.append([i,sim,strtxt])
    with open('../data/sim.txt','w') as fw: #返回最高相似度结果
        lists=[str(line)+'\n' for line in simItems]
        fw.writelines(lists)